# CSII 2024 Exercise 09: Separation Principle, LQG, Stability Margins
&copy; 2024 ETH Zurich, Timm Grigat, Suno Dieckmann, Dejan Milojevic, Niclas Scheuer, Roy Werder; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

## Description
This week's Jupyter notebook will include the small-gain theorem, the stability of feedback control schemes, and performance robustness. 

In [1]:
%pip install C:\CS2solutions\cs2solutions\ --force-reinstall

Note: you may need to restart the kernel to use updated packages.Processing c:\cs2solutions\cs2solutions
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached scipy-1.13.0-cp310-cp310-win_amd64.whl (46.2 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Using cached control-0.10.0-py3-none-any.whl (513 kB)
  Using cached matplotlib-3.8.4-cp310-cp310-win_amd64.whl (7.7 MB)
  Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)
  Using cached fonttools-4.51.0-cp310-cp310-win_amd64.whl (2.2 MB)
  Using cached pillow-10.3.0-cp3


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\nicla\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from cs2solutions import inf_pkg
import control as ct
import numpy as np

np.set_printoptions(suppress=True, precision=3)

### $H^\infty$ norm

The $H^\infty$ norm that we will use to check
the stability of our systems is defined below. The $H^\infty$ norm essentially looks for the highest singular value across all frequencies.

In [3]:
def systemInfnorm(system: ct.TransferFunction) -> float:
    """
    Returns an approximation of the infinity norm of the system

    Parameters:
    - ``system`` (ct.TransferFunction): The system to compute the infinity norm of

    Returns:
    - float: The infinity norm of the system
    """
    # Create a range of frequencies to analyze over
    omega = np.linspace(-4, 4, 1000)
    H = system(omega * 1j)

    # Consider the MIMO case
    if system.ninputs > 1 or system.noutputs > 1:
        # Calculate singular values
        singular_values = [np.linalg.svd(H[..., i])[1] for i in range(len(omega))]
    # Consider the SISO case
    else:
        singular_values = [np.absolute(H[..., i]) for i in range(len(omega))]

    # Return the highest singular value
    return np.vstack(singular_values).max()

# Exercise 1: Small-gain theorem

Consider the following system with three transfer functions: $\alpha, P1, P2$. The functions $P1, P2$ can be accessed using ``inf_pkg.SISO(num)`` as demonstrated below.

In [4]:
tf1 = inf_pkg.SISO(1)
print(tf1)

tf2 = inf_pkg.SISO(2)
print(tf2)

<TransferFunction>: sys[0]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


  1
-----
s + 1

<TransferFunction>: sys[1]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


  1
-----
s - 1



Write a function that checks that all conditions of the small-gain theorem are fulfilled. You should also implement the function __is_stable__ that checks whether a given transfer function is stable.

<img src=./images/block_diagram_0.png alt="Image" width="600" height="200"> 

In [5]:
alpha = 0.5

def is_stable(system: ct.TransferFunction) -> bool:
    """
    Returns whether the system is stable

    Parameters:
    - ``system`` (ct.TransferFunction): The system to check for stability

    Returns:
    - bool: Whether the system is stable
    """
    #TODO
    return False

def small_gain_theorem(systems: list[ct.TransferFunction]) -> bool:
    """
    Checks if the small gain theorem is satisfied for the given systems.

    Parameters:
    - ``systems`` (list[ct.TransferFunction]): The systems to check the small gain theorem for

    Returns:
    - bool: Whether the small gain theorem is satisfied
    """
    #TODO
    return False

In [6]:
Palpha = ct.TransferFunction([alpha], [1])
P1 = inf_pkg.SISO(1)
P2 = inf_pkg.SISO(2)

print(small_gain_theorem([Palpha, P1, P2]))

False


### Testing your functions
In the following cell you can test the implemented function.
To do so simply change the value for $\alpha$.$\\$
For which $\alpha$ will the small gain theorem be fulfilled?$\\$
You can also play around with the systems $P_0$ and $P_1$.
You can choose from the following systems:
$\\
P_0 = \frac{1}{s^2+2s+4} \\
P_1 = \frac{1}{s+1}  \\
P_2 = \frac{1}{s-1}  \\
P_3 = \frac{s-1}{s^2+4s+9} \\
P_4 = 5\frac{s+1}{s+1}  \\
P_5 = 11\frac{s+1}{s-1}  \\
P_6 = 2\frac{s^2-1}{s^2+4s+9} \\
$
For which of the given systems can the small gain theorem not be used? Why?

In [7]:
alpha = 0.5
Palpha = ct.TransferFunction([alpha], [1])
P3 = inf_pkg.SISO(3)
P4 = inf_pkg.SISO(4)
# P5 = ...

print(small_gain_theorem([Palpha, P3, P4]))

False


# Exercise 2: Stability
In this exercise you will create a function called ``internal_stability_check(topbranch, bottombranch)`` that will check the internal stability of an interconnection.

Just as in exercise 1, you will use ``inf_pkg.SISO(num)`` to extract various transfer functions from the diagram.

__HINT:__ All the systems are SISO, so the inverse becomes rather simple.

<img src=./images/block_diagram_1.png alt="Image" width="600" height="200"> 

In [ ]:
# The top tf is defined as P0*P1
# The bottom tf is defined as P2

def internal_stability_check(toptf: ct.TransferFunction, bottomtf: ct.TransferFunction) -> bool:
    """
    Checks if the internal stability condition is satisfied for the given systems.

    Parameters:
    - ``toptf`` (ct.TransferFunction): The top transfer function
    - ``bottomtf`` (ct.TransferFunction): The bottom transfer function
    
    Returns:
    - bool: Whether the internal stability condition is satisfied
    """
    #TODO
    return False

In [ ]:
P0 = inf_pkg.SISO(0)
P1 = inf_pkg.SISO(1)
P2 = inf_pkg.SISO(2)

print(internal_stability_check(P0*P1, P2))

### Testing your function
You can test your function on the systems below:
$\\
P_0 = \frac{1}{s^2+2s+4} \\
P_1 = \frac{1}{s+1}  \\
P_2 = \frac{1}{s-1}  \\
P_3 = \frac{s-1}{s^2+4s+9} \\
P_4 = 5\frac{s+1}{s+1}  \\
P_5 = 11\frac{s+1}{s-1}  \\
P_6 = 2\frac{s^2-1}{s^2+4s+9} \\
$

In [ ]:
P3 = inf_pkg.SISO(3)
P4 = inf_pkg.SISO(4)
# P5 = ...

print(small_gain_theorem([P3, P4]))

### Bonus Exercise

Try to find out experimentally whether ``small_gain_theorem`` or ``internal_stability_check`` is a more __rigorous__ test for feedback stability. You can use all aforementioned Python functions and transfer functions.

# Exercise 3: Stability Robustness
Below we check the robustness of a feedback loop with multiplicative uncertainty.

<img src=./images/block_diagram_2.png alt="Image" width="600" height="200">

In order to use the small-gain theorem, the system is transformed into the following form:
.
$\\
G(s) = -(I+P_2P_1)^{-1}P_2P_1W
\\$

<img src=./images/block_diagram_3.png alt="Image" width="600" height="200">


Apply the small gain theorem to check if the system is robust to the given perturbation and frequency weight. You can use all the functions defined earlier.

In [11]:
Ptriangle = ct.TransferFunction([1], [1, 1])
W = ct.TransferFunction([1], [1, 1])
P1 = inf_pkg.SISO(1)
P2 = 0 #TODO

G = 0 #TODO

# TODO: Print the result